# How to Transform Data Extracted from Wikipedia into a Map

## 1 Data Extraction

I exploit `selenium` to extract the list of Jewish communities.

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options  

def extract_list(url, xpath):
    options = Options()  
    options.add_argument("--headless") 
    options.add_argument("--lang=it");
    driver = webdriver.Chrome(options=options)


    driver.get(url)
    table = []

    # get the list of terms
    words = driver.find_element_by_xpath(xpath).text
    table.extend(words.split('\n'))
    driver.close()
    return table

In [5]:
import pandas as pd 

pages = ['Comunit%C3%A0_ebraiche_italiane', 'Cimiteri_ebraici_in_Italia', 'Musei_ebraici_in_Italia','Ghetti_ebraici_in_Italia','Sinagoghe_in_Italia']
df_dict = {}
xpath = '//*[@id="mw-content-text"]'
table = {}
base_url = 'https://it.wikipedia.org/wiki/'
for page in pages:
    name = page.replace('_', ' ').title().replace('%C3%A0', 'à')
    print(name)
    url = base_url + page
    table[page] = extract_list(url,xpath)
    df_dict[page] = pd.DataFrame(table[page], columns=['value'])
    df_dict[page]['category'] = name

Comunità Ebraiche Italiane
Cimiteri Ebraici In Italia
Musei Ebraici In Italia
Ghetti Ebraici In Italia
Sinagoghe In Italia


In [6]:
df = pd.DataFrame(df_dict[pages[0]])
for i in range(1,len(pages)):
    df = pd.concat([df, df_dict[pages[i]]])
df.tail(10)

,value,category
209,"^ Cfr. alla voce ""Senigallia"" su Italia Judaica",Sinagoghe In Italia
210,"^ Cfr. alla voce ""Urbino"" su Italia Judaica",Sinagoghe In Italia
211,^ http://palermo.repubblica.it/societa/2017/01...,Sinagoghe In Italia
212,Voci correlate[modifica | modifica wikitesto],Sinagoghe In Italia
213,Comunità ebraiche italiane,Sinagoghe In Italia
214,Ghetti ebraici in Italia,Sinagoghe In Italia
215,Musei ebraici in Italia,Sinagoghe In Italia
216,Cimiteri ebraici in Italia,Sinagoghe In Italia
217,Portale Architettura,Sinagoghe In Italia
218,Portale Ebraismo,Sinagoghe In Italia


In [7]:
df.shape

(826, 2)

In [8]:
df.to_csv('data/raw_data.csv')

## 2 Data Cleaning

In [104]:
import pandas as pd

df = pd.read_csv('data/raw_data.csv')
df.head()

,Unnamed: 0,value,category
0,0,La presenza di comunità ebraiche italiane è at...,Comunità Ebraiche Italiane
1,1,Indice,Comunità Ebraiche Italiane
2,2,1 Comunità ebraiche attive,Comunità Ebraiche Italiane
3,3,1.1 Italia settentrionale,Comunità Ebraiche Italiane
4,4,1.2 Italia centrale (e Sardegna),Comunità Ebraiche Italiane


Split communities in active and not active

In [105]:
index_na = 83
index_na

83

In [106]:
def is_active(x, index_na):
    if x < index_na:
        return True
    return False

In [107]:
df['is_active'] = df['Unnamed: 0'].apply(lambda x: is_active(x, index_na))

In [108]:
df.head()

,Unnamed: 0,value,category,is_active
0,0,La presenza di comunità ebraiche italiane è at...,Comunità Ebraiche Italiane,True
1,1,Indice,Comunità Ebraiche Italiane,True
2,2,1 Comunità ebraiche attive,Comunità Ebraiche Italiane,True
3,3,1.1 Italia settentrionale,Comunità Ebraiche Italiane,True
4,4,1.2 Italia centrale (e Sardegna),Comunità Ebraiche Italiane,True


### 2.1 Remove records which do not contain localities

In [109]:
bag_words = ['Comunità ebraica di', '(Provinc', '(Region', ' Provinc', 'ex circondario', 'Cimitero ebraico di', 'Museo ebraico di', 'Ghetto di', 'Sinagoga di', 'Cimitero israelitico di', 'Cimitero monumentale ebraico di']
def is_locality(x):
    for bag in bag_words:
        if bag in x:
            return True
    return False

In [110]:
df['is_locality'] = df['value'].apply(lambda x : is_locality(x))

In [111]:
df = df[df['is_locality'] == True]
df.reset_index(inplace = True)
df.head(10)

,index,Unnamed: 0,value,category,is_active,is_locality
0,35,35,"Bologna (Provincia di Bologna), 220",Comunità Ebraiche Italiane,True,True
1,36,36,"Casale Monferrato (ex circondario di Casale), ...",Comunità Ebraiche Italiane,True,True
2,37,37,"Ferrara (Province di Ferrara, Forlì-Cesena e R...",Comunità Ebraiche Italiane,True,True
3,38,38,"Genova (Regione Liguria), 650-700ca.",Comunità Ebraiche Italiane,True,True
4,40,40,"Mantova (Province di Bergamo, Brescia, Cremona...",Comunità Ebraiche Italiane,True,True
5,41,41,"Merano (Regione Trentino Alto Adige), 50-60",Comunità Ebraiche Italiane,True,True
6,42,42,"Milano (Province di Como, Milano, Pavia, Sondr...",Comunità Ebraiche Italiane,True,True
7,43,43,"Modena (Province di Modena e Reggio Emilia), 114",Comunità Ebraiche Italiane,True,True
8,45,45,"Padova (Province di Padova e Rovigo), 200",Comunità Ebraiche Italiane,True,True
9,46,46,"Parma (Province di Parma e Piacenza), 21",Comunità Ebraiche Italiane,True,True


### 2.2 Identify delimiters within the text to extract the exact locality.

In [112]:
start_bag = ['Comunità ebraica di','Sinagoga di', 'Cimitero ebraico di','Sinagoghe del Ghetto di','. Getto di','Ghetto di','Cimitero monumentale ebraico di','Cimitero israelitico di', 'Museo ebraico di']
stop_bag = ['oratorio', '[', ',', 'aperta', 'viale', 'presso', '-', 'di rito','(']

In [117]:
def get_start_index(x):
    start_index = 0
    for bag in start_bag:
        if bag in x:
            return len(bag)
    return start_index

In [98]:
def get_stop_index(x):
    min_index = len(x)
    for stop in stop_bag:
        try:
            curr_index = x.index(stop)
            if curr_index < min_index:
                min_index = curr_index
        except ValueError:
            continue
    return min_index

In [114]:
def get_locality(x):
    if 'Fiuggi' in x:
        return 'Anticoli'
    if x == 'Cimitero ebraico di viale Ippolito Nievo a Livorno (dismesso)':
        return 'Livorno'
    if x == 'Sinagoga di via dei Ramaglianti (Firenze), scomparsa':
        return 'Firenze'
    start_index = get_start_index(x)
    stop_index = get_stop_index(x)  
    return x[start_index:stop_index].strip()
    

In [115]:
df['locality'] = df['value'].apply(lambda x: get_locality(x))

In [116]:
df.to_csv('data/data_locality.csv')

## 3 Data Enrichment through Geocoding

In [122]:
from geopy.geocoders import Nominatim

def get_coordinates(city_list):
    geolocator = Nominatim(user_agent="location script")
    dicto = {}
    
    for city in city_list:
        print(city)
        try:
            location = geolocator.geocode(city, country_codes='it')
        except:
            raise Exception("There was a problem with the getCoordinates function")
        coordinate_values = (location.longitude, location.latitude)  
        dicto[city] = coordinate_values 
    return dicto 

In [123]:
city_coords_dict = get_coordinates(df['locality'].values)

Bologna
Casale Monferrato
Ferrara
Genova


Exception: There was a problem with the getCoordinates function

In [ ]:
def get_coords(x, city_coords_dict, coord = 0):
    coords = city_coords_dict[x]
    return coords[coord]

In [ ]:
df['longitude'] = df['locality'].apply(lambda x : get_coords(x, city_coords_dict))
df['latitude'] = df['locality'].apply(lambda x : get_coords(x, city_coords_dict, coord = 1))

Save Results

In [ ]:
df.to_csv('data/geo_data.csv')

## 4 Data Viz on a Map

In [241]:
categories = []
for page in pages:
    category = page.replace('_', ' ').title().replace('%C3%A0', 'à')
    categories.append(category)
categories

['Comunità Ebraiche Italiane',
 'Cimiteri Ebraici In Italia',
 'Musei Ebraici In Italia',
 'Ghetti Ebraici In Italia',
 'Sinagoghe In Italia']

In [242]:
import folium
my_map = folium.Map(tiles='cartodb positron',location=[41.8719, 12.5674],zoom_start=6)

In [243]:
def add_markers_to_the_map(the_map, df, icon, category,color):  
    group = folium.FeatureGroup(name=category).add_to(the_map)
    for i in range(0, df.shape[0]):    
            popup_text = df.iloc[i]['value']
            city = df.iloc[i]['locality']
            long = df.iloc[i]['longitude']
            lat = df.iloc[i]['latitude']
            
            popup = folium.Popup(popup_text, autopan='False', parse_html=True)
                
            marker = folium.Marker(location=[lat, long], 
                                   popup=popup, 
                                   icon = folium.Icon(icon_size=(25, 25), color=color, icon=icon, prefix='fa'))
            
            group.add_child(marker)
            
    return the_map

In [244]:
colors = ['green', 'blue', 'orange', 'brown', 'pink']
color = 0
for category in categories:
    df_selected = df[df['category'] == category]
    if category == 'Comunità Ebraiche Italiane':
        add_markers_to_the_map(my_map, df_selected[df['is_active'] == True], 'flag', category + ' Attive', 'green')
        add_markers_to_the_map(my_map, df_selected[df['is_active'] == False], 'flag', category + ' Non più Attive', 'red')
    else:
        add_markers_to_the_map(my_map, df_selected, 'flag', category, colors[color])
    color = color + 1
folium.LayerControl().add_to(my_map)

<ipython-input-244-064f05806e90>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  add_markers_to_the_map(my_map, df_selected[df['is_active'] == True], 'flag', category + ' Attive', 'green')
<ipython-input-244-064f05806e90>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  add_markers_to_the_map(my_map, df_selected[df['is_active'] == False], 'flag', category + ' Non più Attive', 'red')
<ipython-input-243-b28382cfd214>:13: UserWarning: color argument of Icon should be one of: {'green', 'purple', 'pink', 'white', 'orange', 'lightred', 'darkred', 'black', 'blue', 'lightblue', 'beige', 'darkblue', 'darkpurple', 'gray', 'cadetblue', 'lightgreen', 'darkgreen', 'lightgray', 'red'}.
  icon = folium.Icon(icon_size=(25, 25), color=color, icon=icon, prefix='fa'))


In [245]:
my_map.save('mappa.html')

Since folium does not support mouseover/mouseout, modify the html file, to support it.

In [246]:
import re
import fileinput

with open("mappa.html") as inf:
    txt = inf.read()

#Find all the markers names given by folium
markers = re.findall(r'\bmarker_\w+', txt)
markers = list(set(markers))


for marker in markers:
    for linenum,line in enumerate( fileinput.FileInput("mappa.html",inplace=1) ):
        pattern = marker + ".bindPopup"
        pattern2 = marker + ".on('mouseover', function (e) {this.openPopup();});"
        pattern3 = marker + ".on('mouseout', function (e) {this.closePopup();});"
    
    

        if pattern in line:
            print(line.rstrip())
            print(pattern2)
            print(pattern3)
        else:
            print(line.rstrip())